# Part 1: Tool Calling 101



Tool calling is one of the most important aspects when building with agents. In this part of the tutorial I'll show you how to implement simple tool/function calling with DeepSeek


First, we setup credentials: 

In [ ]:
import os
from dotenv import load_dotenv
import jupyter_black
from openai import OpenAI
from typing import Any
import inspect
from pydantic import BaseModel


jupyter_black.load()

load_dotenv(dotenv_path="api_keys.env")

API_KEY = os.getenv("DEEPSEEK_API_KEY")
BASE_URL = "https://api.deepseek.com"
MODEL = "deepseek-chat"
client = OpenAI(api_key=API_KEY, base_url=BASE_URL)

We first create a function to call our llm: 

In [2]:
def send_messages(messages, model=MODEL, tools=None):
    response = client.chat.completions.create(
        model=model, messages=messages, tools=tools, temperature=0.0
    )
    return response.choices[0].message

And we can use it to call our LLM 

In [3]:
print(
    send_messages(
        messages=[{"role": "user", "content": "How is the weather in Lisbon?"}],
    ).content
)

To check the current weather in Lisbon, you can use reliable weather services like **Weather.com (The Weather Channel), AccuWeather, or Meteoblue**. Here’s a general idea of Lisbon’s climate:  

- **Summer (June–September):** Warm and sunny, with temperatures around **25–30°C (77–86°F)**, sometimes reaching **35°C (95°F)** in heatwaves.  
- **Autumn (October–November):** Mild, with occasional rain, temperatures **15–23°C (59–73°F)**.  
- **Winter (December–February):** Cool and rainy, averaging **8–15°C (46–59°F)**, rarely below freezing.  
- **Spring (March–May):** Pleasant, gradually warming, with temperatures **12–22°C (54–72°F)**.  

For **real-time updates**, I recommend checking:  
- [Weather.com (Lisbon)](https://weather.com)  
- [AccuWeather (Lisbon)](https://www.accuweather.com)  
- [IPMA (Portuguese Institute)](https://www.ipma.pt)  

Would you like a forecast for a specific date? 😊


Now, let's create a function to fetch the weather: 

In [4]:
class WeatherRequest(BaseModel):
    city: str


def get_weather(request: WeatherRequest) -> str:
    return f"The weather in {request.city} is sunny"


def to_openai_tool(func: callable) -> dict[str, Any]:
    sig = inspect.signature(func)
    if sig.parameters:
        param = next(iter(sig.parameters.values()))
        param_type = param.annotation
        schema = param_type.model_json_schema()
    else:
        param_type = None
        schema = {"type": "object", "properties": {}, "required": []}

    return {
        "type": "function",
        "function": {
            "name": func.__name__,
            "description": (param_type.__doc__ if param_type else func.__doc__ or ""),
            "parameters": schema,
        },
    }

And we send that tool as well to DeepSeek: 

In [5]:
tools = [to_openai_tool(get_weather)]
messages = [{"role": "user", "content": "How is the weather in Lisbon?"}]

message = send_messages(
    messages=messages,
    tools=tools,
)
messages.append(message)

print("User message: ", messages[0]["content"])

tool = message.tool_calls[0]
print("Tool call: ", tool)

User message:  How is the weather in Lisbon?
Tool call:  ChatCompletionMessageToolCall(id='call_0_5ced15ad-23f6-4a40-8aff-684bf5027f5b', function=Function(arguments='{"city":"Lisbon"}', name='get_weather'), type='function', index=0)


And we get the result for that tool: 

In [6]:
tool_name = tool.function.name
tool_args = tool.function.arguments
tool_result = globals()[tool_name](WeatherRequest.model_validate_json(tool_args))
tool_call_id = tool.id

print(tool_name)
print(tool_args)
print(tool_result)
print(tool_call_id)

get_weather
{"city":"Lisbon"}
The weather in Lisbon is sunny
call_0_5ced15ad-23f6-4a40-8aff-684bf5027f5b


Finally, we send that result back to our LLM: 

In [7]:
messages.append(
    {
        "role": "tool",
        "tool_call_id": tool.id,
        "content": tool_result,
    }
)
message = send_messages(messages)

In [8]:
print(message.content)

The weather in Lisbon is currently sunny. Enjoy the sunshine! ☀️


## **Exercise**: 
- Create a tool call to fetch the user's name and age and make the model use it. 

In [ ]:
def send_messages(messages, model=MODEL, tools=None):
    response = client.chat.completions.create(
        model=model, messages=messages, tools=tools, temperature=0.0
    )
    return response.choices[0].message


def get_user_info() -> str:
    # 1. Implement this function. What information do we need about the user?
    # Tip: Why not use the input function?
    return "I don't know"


tools = [to_openai_tool(get_user_info)]

messages = [{"role": "user", "content": "What is my name and age?"}]
message = send_messages(messages, tools=tools)
print(f"User message: {messages[0]['content']}")

tool = message.tool_calls[0]
tool_args = tool.function.arguments
tool_id = tool.id
tool_name = tool.function.name
messages.append(message)

tool_result = globals()[tool_name]()

messages.append({"role": "tool", "tool_call_id": tool.id, "content": tool_result})
message = send_messages(messages)
print(f"Model>\t {message.content}")

Great! Now that you know about tool calling! It's time to learn about agents vs. workflows!